In [3]:
!pip install vecstack

In [4]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [5]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [6]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

trainfile = r'/gdrive/My Drive/CIS508/ASN3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS508/ASN3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]
(173836, 596)
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...                     0                 IL
1                 4                 5  ...                     0                 NJ
2                 3                 3  ...                     0                 NJ
3                 5                 9  ...                     0                 TX
4                12                21  ...                     0                 CA

[5 rows x 596 columns]


In [7]:
#Copy Train data excluding target
trainData_Copy = train_data.iloc[:,:-1].copy()
#Copy Test data excluding GeographicalField64 column
testData_Copy = test_data.iloc[:,:-1].copy()

#Separate Train data and test data
X_train = trainData_Copy
X_test = testData_Copy
#Selecting only target columm from Training dataset
y_train=train_data.iloc[:,-1] 




print(X_train.head())
print(X_test.head()) 
print(y_train.head())
 

   CoverageField11A  ...  GeographicField64_TX
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     1
4                 4  ...                     0

[5 rows x 595 columns]
   CoverageField11A  ...  GeographicField64_TX
0                13  ...                     0
1                 4  ...                     0
2                 3  ...                     0
3                 5  ...                     0
4                12  ...                     0

[5 rows x 595 columns]
0    0
1    0
2    0
3    0
4    0
Name: QuoteConversion_Flag, dtype: int64


In [8]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict_train=clf.predict(X_train)
print("accuracy Score (training) for Decision Tree:{0:6f}".format(clf.score(X_train,y_train)))
print("Confusion Matrix for training Decision Tree")
print(confusion_matrix(y_train,clf_predict_train))
print("Classification report for training Decision Tree")
print(classification_report(y_train,clf_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(clf.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction and corresponding probability
output_col.to_csv("/gdrive/My Drive/CIS508/ASN3/DefaultDecisionTree_Test_results.csv", index = None)

accuracy Score (training) for Decision Tree:1.000000
Confusion Matrix for training Decision Tree
[[52738     0]
 [    0 12262]]
Classification report for training Decision Tree
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     52738
           1       1.00      1.00      1.00     12262

    accuracy                           1.00     65000
   macro avg       1.00      1.00      1.00     65000
weighted avg       1.00      1.00      1.00     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     1


In [9]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

{'min_samples_split': 30, 'max_depth': 7}


In [10]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,y_train)
clf_predict_train=clf.predict(X_train)
print("accuracy Score (training) for Decision Tree:{0:6f}".format(clf.score(X_train,y_train)))
print("Confusion Matrix for training Decision Tree")
print(confusion_matrix(y_train,clf_predict_train))
print("Classification report for training Decision Tree")
print(classification_report(y_train,clf_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(clf.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())



accuracy Score (training) for Decision Tree:0.912338
Confusion Matrix for training Decision Tree
[[52213   525]
 [ 5173  7089]]
Classification report for training Decision Tree
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     52738
           1       0.93      0.58      0.71     12262

    accuracy                           0.91     65000
   macro avg       0.92      0.78      0.83     65000
weighted avg       0.91      0.91      0.90     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [34]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict_train=rfc.predict(X_train)
print("accuracy Score (training) for Random Forest:{0:6f}".format(rfc.score(X_train,y_train)))
print("Confusion Matrix for training Random Forest")
print(confusion_matrix(y_train,rfc_predict_train))
print("Classification report for training Random Forest")
print(classification_report(y_train,rfc_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(rfc.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())





accuracy Score (training) for Random Forest:0.992431
Confusion Matrix for training Random Forest
[[52720    18]
 [  474 11788]]
Classification report for training Random Forest
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     52738
           1       1.00      0.96      0.98     12262

    accuracy                           0.99     65000
   macro avg       0.99      0.98      0.99     65000
weighted avg       0.99      0.99      0.99     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [35]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,300,20),'min_samples_split' : range(5,50,10),'max_depth': range(5,50,38)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

KeyboardInterrupt: ignored

In [13]:


#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,y_train)
rfc_predict_train=rfc.predict(X_train)
print("accuracy Score (training) for Random Forest:{0:6f}".format(rfc.score(X_train,y_train)))
print("Confusion Matrix for training Random Forest")
print(confusion_matrix(y_train,rfc_predict_train))
print("Classification report for training Random Forest")
print(classification_report(y_train,rfc_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(rfc.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction
output_col.to_csv("/gdrive/My Drive/CIS508/ASN3/RandomForestHyper_beforeTuning_Test_results.csv", index = None)




accuracy Score (training) for Random Forest:0.946185
Confusion Matrix for training Random Forest
[[52587   151]
 [ 3347  8915]]
Classification report for training Random Forest
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     52738
           1       0.98      0.73      0.84     12262

    accuracy                           0.95     65000
   macro avg       0.96      0.86      0.90     65000
weighted avg       0.95      0.95      0.94     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [14]:
#K_Nearest Neighbour============================================================

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
knn_predict_train=knn.predict(X_train)
print("accuracy Score (training) for KNN:{0:6f}".format(knn.score(X_train,y_train)))
print("Confusion Matrix for training KNN")
print(confusion_matrix(y_train,knn_predict_train))
print("Classification report for training KNN")
print(classification_report(y_train,knn_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as QuoteConversion_Flag
pred=pd.DataFrame(knn.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction
output_col.to_csv("/gdrive/My Drive/CIS508/ASN3/KNNHyper_beforeTuning_Test_results.csv", index = None)

accuracy Score (training) for KNN:0.829215
Confusion Matrix for training KNN
[[51551  1187]
 [ 9914  2348]]
Classification report for training KNN
              precision    recall  f1-score   support

           0       0.84      0.98      0.90     52738
           1       0.66      0.19      0.30     12262

    accuracy                           0.83     65000
   macro avg       0.75      0.58      0.60     65000
weighted avg       0.81      0.83      0.79     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [15]:
#SVM Classifier============================================================

svmc = LinearSVC() # Linear Kernel
svmc.fit(X_train,y_train)
svmc_predict_train=svmc.predict(X_train)
print("accuracy Score (training) for SVM:{0:6f}".format(svmc.score(X_train,y_train)))
print("Confusion Matrix for training SVM")
print(confusion_matrix(y_train,svmc_predict_train))
print("Classification report for training SVM")
print(classification_report(y_train,svmc_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as QuoteConversion_Flag
pred=pd.DataFrame(svmc.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction
output_col.to_csv("/gdrive/My Drive/CIS508/ASN3/SVMBeforeTuning_Test_results.csv", index = None)

accuracy Score (training) for SVM:0.421400
Confusion Matrix for training SVM
[[16702 36036]
 [ 1573 10689]]
Classification report for training SVM
              precision    recall  f1-score   support

           0       0.91      0.32      0.47     52738
           1       0.23      0.87      0.36     12262

    accuracy                           0.42     65000
   macro avg       0.57      0.59      0.42     65000
weighted avg       0.78      0.42      0.45     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     1
1            5                     0
2            7                     1
3            9                     0
4           10                     1


In [16]:
#Gradient Boosting ============================================================

abc =GradientBoostingClassifier()
abc.fit(X_train,y_train)
abc_predict_train=abc.predict(X_train)
print("accuracy Score (training) for Gradient Boosting:{0:6f}".format(abc.score(X_train,y_train)))
print("Confusion Matrix for training Gradient Boosting")
print(confusion_matrix(y_train,abc_predict_train))
print("Classification report for training Gradient Boosting")
print(classification_report(y_train,abc_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as QuoteConversion_Flag
pred=pd.DataFrame(abc.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())


accuracy Score (training) for Gradient Boosting:0.923554
Confusion Matrix for training Gradient Boosting
[[52114   624]
 [ 4345  7917]]
Classification report for training Gradient Boosting
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     52738
           1       0.93      0.65      0.76     12262

    accuracy                           0.92     65000
   macro avg       0.92      0.82      0.86     65000
weighted avg       0.92      0.92      0.92     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [36]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.6)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 31642})


In [37]:
#STACKING MODELS =====================================================================
print("_________________________________________________________________________s\nEnsemble Methods Predictions using RandomForest,KNN and SVM Classifier\n")

models = [ RandomForestClassifier(),KNeighborsClassifier(),LinearSVC() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

_________________________________________________________________________s
Ensemble Methods Predictions using RandomForest,KNN and SVM Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestClassifier]
    fold  0:  [0.91742510]
    fold  1:  [0.91462837]
    fold  2:  [0.91732246]
    fold  3:  [0.91689580]
    ----
    MEAN:     [0.91656793] + [0.00113726]
    FULL:     [0.91656791]

model  1:     [KNeighborsClassifier]
    fold  0:  [0.69534509]
    fold  1:  [0.69852105]
    fold  2:  [0.69944060]
    fold  3:  [0.69593249]
    ----
    MEAN:     [0.69730981] + [0.00171497]
    FULL:     [0.69730979]

model  2:     [LinearSVC]
    fold  0:  [0.62689609]
    fold  1:  [0.65149791]
    fold  2:  [0.38451692]
    fold  3:  [0.64767232]
    ----
    MEAN:     [0.57764581] + [0.11189527]
    FULL:     [0.57764873]



In [29]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.4)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))


___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 21095})


In [30]:
#STACKING MODELS =====================================================================
print("_________________________________________________________________________s\nEnsemble Methods Predictions using RandomForest,KNN and SVM Classifier\n")

models = [ RandomForestClassifier(),KNeighborsClassifier(),LinearSVC() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=2, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


_________________________________________________________________________s
Ensemble Methods Predictions using RandomForest,KNN and SVM Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestClassifier]
    fold  0:  [0.89804155]
    fold  1:  [0.90034132]
    ----
    MEAN:     [0.89919143] + [0.00114988]
    FULL:     [0.89919142]

model  1:     [KNeighborsClassifier]
    fold  0:  [0.69382669]
    fold  1:  [0.69647307]
    ----
    MEAN:     [0.69514988] + [0.00132319]
    FULL:     [0.69514987]

model  2:     [LinearSVC]
    fold  0:  [0.59157028]
    fold  1:  [0.45747102]
    ----
    MEAN:     [0.52452065] + [0.06704963]
    FULL:     [0.52452156]



In [26]:
#STACKING - CONSTRUCT A KNN MODEL==============================
model =  KNeighborsClassifier(n_neighbors=5)
    
model = model.fit(S_Train, y_res)
y_pred=model.predict(S_Train)
print("accuracy Score (training) for KNN after STACKING:{0:6f}".format(model.score(S_Train,y_res)))
print("Confusion Matrix for training KNN after STACKING")
print(confusion_matrix(y_res,y_pred))
print("Classification report for training KNN after STACKING")
print(classification_report(y_res,y_pred))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as QuoteConversion_Flag
pred=pd.DataFrame(model.predict(S_Test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction 
output_col.to_csv("/gdrive/My Drive/CIS508/ASN3/KNNAfterStacking_Test_results.csv", index = None)


ValueError: ignored

In [31]:
#STACKING - RANDOM FOREST MODEL AFTER TUNING==============================
model =  RandomForestClassifier(**grid_parm_rfc)
    
model = model.fit(S_Train, y_res)
y_pred=model.predict(S_Train)
print("accuracy Score (training) for RANDOM FOREST after STACKING:{0:6f}".format(model.score(S_Train,y_res)))
print("Confusion Matrix for training RANDOM FOREST after STACKING")
print(confusion_matrix(y_res,y_pred))
print("Classification report for training RANDOM FOREST after STACKING")
print(classification_report(y_res,y_pred))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as QuoteConversion_Flag
pred=pd.DataFrame(model.predict(S_Test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction
output_col.to_csv("/gdrive/My Drive/CIS508/ASN3/RandomForestAfterStacking3_Test_results.csv", index = None)

accuracy Score (training) for RANDOM FOREST after STACKING:0.899191
Confusion Matrix for training RANDOM FOREST after STACKING
[[51280  1458]
 [ 5985 15110]]
Classification report for training RANDOM FOREST after STACKING
              precision    recall  f1-score   support

           0       0.90      0.97      0.93     52738
           1       0.91      0.72      0.80     21095

    accuracy                           0.90     73833
   macro avg       0.90      0.84      0.87     73833
weighted avg       0.90      0.90      0.90     73833

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [21]:
#STACKING - SVC MODEL AFTER TUNING==============================
model =  LinearSVC()
    
model = model.fit(S_Train, y_res)
y_pred=model.predict(S_Train)
print("accuracy Score (training) for SVM after STACKING:{0:6f}".format(model.score(S_Train,y_res)))
print("Confusion Matrix for training SVM after STACKING")
print(confusion_matrix(y_res,y_pred))
print("Classification report for training RANDOM FORESTSVM after STACKING")
print(classification_report(y_res,y_pred))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as QuoteConversion_Flag
pred=pd.DataFrame(model.predict(S_Test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction
output_col.to_csv("/gdrive/My Drive/CIS508/ASN3/SVMAfterStacking_Test_results.csv", index = None)

accuracy Score (training) for SVM after STACKING:0.909616
Confusion Matrix for training SVM after STACKING
[[51205  1533]
 [ 5617 20752]]
Classification report for training RANDOM FORESTSVM after STACKING
              precision    recall  f1-score   support

           0       0.90      0.97      0.93     52738
           1       0.93      0.79      0.85     26369

    accuracy                           0.91     79107
   macro avg       0.92      0.88      0.89     79107
weighted avg       0.91      0.91      0.91     79107

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0
